In [1]:
from typing import Optional

import sqlalchemy
from prorrataerv.model.pcp_solution import models
from prorrataerv.model.pcp_solution import queries


class Querier_2:
    def __init__(self, conn: sqlalchemy.engine.Connection):
        self._conn = conn

    def get_nodes_generator(
        self, *, category_ids: set[int]
    ) -> Optional[list[models.NodesGeneratorSchema]]:
        #params = [{"category_ids": ids} for ids in category_ids]
        #print(params)
        query = sqlalchemy.text(queries.GET_NODES_GENERATORS).bindparams(sqlalchemy.bindparam("category_ids", expanding=True))
        result = self._conn.execute(
            query, {"category_ids": category_ids}
        ).all()

        if result is None:
            return None

        return result


In [2]:
from pathlib import Path

path_prg = Path(r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20241007\Datos\Model PRGdia_Full_Definitivo Solution\Model PRGdia_Full_Definitivo Solution.accdb")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = sqlalchemy.engine.URL.create(
    "access+pyodbc", query={"odbc_connect": connection_string}
)

eng = sqlalchemy.create_engine(connection_url)

with eng.connect() as conn:
    store = Querier_2(conn=conn)
    nodes = store.get_nodes_generator(category_ids=[99,100])

In [3]:
nodes

[('ALENA_EO', 'Charrua154'),
 ('ATACAMA_EO', 'Maitencillo220'),
 ('AURORA_EO', 'Tineo220'),
 ('CABO_LEONES_1_EO', 'Maitencillo220'),
 ('CABO_LEONES_2_EO', 'Maitencillo220'),
 ('CABO_LEONES_3_EO', 'Maitencillo220'),
 ('CALAMA_EO', 'Salar220'),
 ('CAMPO_LINDO_EO', 'Mulchen220'),
 ('CANELA_2_EO', 'LPalmas220'),
 ('CANELA_EO', 'LPalmas220'),
 ('CARDONAL_EO', 'Rapel220'),
 ('CERRO_TIGRE_EO', 'Ohiggins220_BP1'),
 ('CUEL_EO', 'Charrua154'),
 ('EL_ARRAYAN_EO', 'DonGoyo220'),
 ('EL_ARREBOL_EO', 'Horcones066'),
 ('EL_MAITEN_EO', 'Charrua220'),
 ('EL_NOGAL_EO', 'Mulchen220'),
 ('HORIZONTE_EO', 'Parinas500'),
 ('HUAJACHE_EO', 'Horcones066'),
 ('KALLPA_EO', 'Parinas500'),
 ('LA_CABANA_EO', 'Mulchen220'),
 ('LA_ESPERANZA_EO', 'Charrua154'),
 ('LA_ESTRELLA_EO', 'Rapel220'),
 ('LA_FLOR_EO', 'Charrua154'),
 ('LAS_PENAS_EO', 'Charrua154'),
 ('LEBU_3_EO', 'Horcones066'),
 ('LEBU_EO', 'Horcones066'),
 ('LLANOS_DEL_VIENTO_EO', 'Ohiggins220_BP1'),
 ('LOMAS_DE_DUQUECO_EO', 'Temuco220'),
 ('LOS_BUENOS_AIRES_E

In [6]:
from prorrataerv.model.pcp_solution import Querier
import sqlalchemy
from pathlib import Path

path_prg = Path(r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20241007\Datos\Model PRGdia_Full_Definitivo Solution\Model PRGdia_Full_Definitivo Solution.accdb")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = sqlalchemy.engine.URL.create(
    "access+pyodbc", query={"odbc_connect": connection_string}
)

eng = sqlalchemy.create_engine(connection_url)

with eng.connect() as conn:
    pcp_solution = Querier(conn=conn)
    nodes = pcp_solution.get_nodes_generator(category_ids=[99,100])


In [11]:
solares = [node.generator for node in nodes if node.generator.endswith("_FV")]
eolicas = [node.generator for node in nodes if node.generator.endswith("_EO")]
eolicas

[]